# Image Offsets: Process sequence

This notebook transforms the procedures developed in `Offsets_1.ipynb` into callable functions. 

These functions are used in a loop to process an entire sequence of images. Results are later examined in plots.

In [1]:
%pylab notebook
%matplotlib notebook

import os, glob

import numpy as np
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

from scipy.optimize import minimize
from astropy.table import Table, vstack
from astropy.stats import SigmaClip

import photutils
from photutils import Background2D, ModeEstimatorBackground, DAOStarFinder

import rawpy

Populating the interactive namespace from numpy and matplotlib


## Initialization

In [2]:
dirpath = '../astrophotography_data/MilkyWayPrettyBoy/12800/light/'
file_list = glob.glob(dirpath + '/*.ARW')

In [3]:
# parameters for background subtraction and star finding
bkg_sigma = 3.0
bkg_cell_footprint = (100, 100)
bkg_filter = (5, 5)
dao_fwhm = 3.0
dao_threshold = 10.

# operators
sigma_clip = SigmaClip(sigma=bkg_sigma)
bkg_estimator = ModeEstimatorBackground()

In [4]:
# 1st test image - this will be the reference image which subsequent images
# will have their offsets computed against. We need to read it here to get 
# the camera color array specification as well.
fname = file_list[0]
raw = rawpy.imread(fname)
ref_imarray = raw.raw_image_visible.astype(float)

In [5]:
# masks that isolate the RGB pixels - these are camera-dependent and work with all images
colors_array = raw.raw_colors_visible

red_mask = np.where(colors_array == 0, 1, 0)

green_mask_1 = np.where(colors_array == 1, 1, 0)
green_mask_2 = np.where(colors_array == 3, 1, 0)
green_mask = green_mask_1 | green_mask_2

blue_mask = np.where(colors_array == 2, 1, 0)

In [6]:
# normalization factors for the RGB arrays
red_norm = 1.321875  # smooth background
green_norm = 1.
blue_norm = 1.27695312

## Function definitios

In [7]:
# computes position offsets between two tables. 
def get_offsets(sources, sources_prev):

    sources.add_column(np.nan, name='xoffset')
    sources.add_column(np.nan, name='yoffset')
    sources.add_column(0, name='ref_row')
    sources.add_column(0, name='prev_row')

    # loop over rows in previous table
    for row_index_prev in range(len(sources_prev)):
        # index in reference table
        ref_row = sources_prev[row_index_prev]['ref_row']

        # if previous table does not contain a pointer to the reference 
        # table, ignore.
        if ref_row == 0:
            continue

        # get position in previous table
        x_prev = sources_prev[row_index_prev]['xcentroid']
        y_prev = sources_prev[row_index_prev]['ycentroid']

        # loop over rows in current table
        for row_index in range(len(sources3)):
            x = sources[row_index]['xcentroid']
            y = sources[row_index]['ycentroid']

            # offsets in relation to previous table - these are the ones to check for proximity
            x_off_previous = x - x_prev
            y_off_previous = y - y_prev

            # check for proximity, and store relevant info if found
            if abs(x_off_previous) <= 1.5 and abs(y_off_previous) <= 1.5:

                # offsets in relation to reference table
                sources[row_index]['xoffset'] = x - sources[ref_row]['xcentroid']
                sources[row_index]['yoffset'] = y - sources[ref_row]['ycentroid']

                # store pointers to rows in reference and previous tables
                sources[row_index]['ref_row'] = ref_row
                sources[row_index]['prev_row'] = row_index_prev

                break # if there is another star that matches the criterion, just ignore it

In [8]:
# creates a table with star positions, given a path to and image file
def find_stars(path):

    with rawpy.imread(path) as raw:
        imarray = raw.raw_image_visible.astype(float)

        # normalize
        raw_norm_1 = imarray * (red_mask * red_norm)
        raw_norm_2 = raw_norm_1 + imarray * (green_mask * 1.0)
        raw_norm = raw_norm_2 + imarray * (blue_mask * blue_norm)

        # handle saturated pixels
        raw_norm = np.where(imarray > 16380, imarray, raw_norm)

        # compute and subtract background
        bkg = Background2D(raw_norm, bkg_cell_footprint, filter_size=bkg_filter, sigma_clip=sigma_clip, bkg_estimator=bkg_estimator)
        subtracted = raw_norm - bkg.background

        # find stars
        daofind = DAOStarFinder(fwhm=dao_fwhm, threshold=dao_threshold * bkg.background_rms_median) 
        sources = daofind(subtracted)

        return sources

In [9]:
# keep only the NaN-free entries
def clean_nans(sources):
    has_nan = np.zeros(len(sources), dtype=bool)
    xoff = np.array(sources['xoffset'])
    has_nan |= np.isnan(xoff)
    return sources[~has_nan]

## Process sequence

In [10]:
# find stars in reference image
sources_ref = find_stars(file_list[0])

# force reference image to be the "previous" image
sources_prev = sources_ref

In [11]:
# loop over list of remaining images
for file_path in file_list[1:3]:
    
    # find stars
    sources = find_stars(file_path)
    
    # compute offsets
    sources_current = get_offsets(sources, sources_prev)
    
    sources_current_no_nan = clean_nans(sources_current)
    
    # save table to file
    
    # for next iteration, current table becomes previous
    sources_prev = sources_current_no_nan

KeyError: 'ref_row'